# Módulo de Generación de Recetas en Quechua

Instalación de Dependencias

In [ ]:
!pip install -U deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 809.5 kB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator
import re
import os

Definimos las funciones de preprocesamiento y para traducir el texto

In [ ]:
def quy_translator(text):
  return GoogleTranslator(source='es', target='qu').translate(text)

In [148]:
def f_obtener_texto(ruta_receta):
    # Read the text from the file
    with open(ruta_receta, 'r') as file:
        text = file.read()

    # Split the text into sections based on the markers
    sections = text.split('\n')

    # Initialize variables to store the sections
    receta = []
    ingredientes = []
    instrucciones = []

    # Iterate through the sections and assign them to the appropriate variables
    current_section = None
    for line in sections:
        if line.startswith('receta'):
            current_section = 'Receta'
            receta.append(line.lstrip('receta ').strip())
            continue
        elif line.startswith('ingredientes'):
            current_section = 'ingredientes'
            first_ing = line.strip('ingredientes').strip()
            if first_ing:
              ingredientes.append()
            continue
        elif line.startswith('instrucciones'):
            current_section = 'instrucciones'
            first_inst = line.strip('instrucciones').strip()
            if first_inst:
              instrucciones.append(first_ing)
            continue

        if current_section == 'receta':
            receta.append(line)
        elif current_section == 'ingredientes':
            ingredientes.append(line)
        elif current_section == 'instrucciones':
            instrucciones.append(line)

    # Return the sections
    return [receta, ingredientes, instrucciones]

def f_traducir_texto(receta, ingredientes, instrucciones):
  receta = [quy_translator(r) for r in receta]
  ingredientes = [quy_translator(i) for i in ingredientes]
  instrucciones = [quy_translator(i) for i in instrucciones]
  return [receta, ingredientes, instrucciones]

Testing f_obtener_texto & f_traducir_texto

In [ ]:
ruta_receta = '/content/recetas/Ajiaco de Olluco.txt'
receta, ingredientes, instrucciones = f_obtener_texto(ruta_receta)
print("Receta:", receta)
print("Ingredientes:", ingredientes)
print("Instrucciones:", instrucciones)

Receta: ['Ajiaco de Olluco']
Ingredientes: ['1 Cebolla Roja', '', 'picada en cuadraditos', '1 Cucharada de Ajo Molido', '1 Cucharada de Palillo', '', 'en polvo', '1 Unidad deMAGGI® Cubo Carne', '1 Cucharada de Aceite Vegetal', '1 Taza de Agua', '3 Tazas de Olluco', '', 'picado', '3 Tajadas de  Queso Fresco', '', 'picado', '1 Lata deIDEAL® CREMOSITA', '2 Huevos', '1 Pizca de Pimienta', '1 Pizca de Comino', '2 Tazas de Arroz cocido']
Instrucciones: ['En una olla pequeña, calentar el aceite y dorar la cebolla, el ajo', 'Luego agregar el MAGGI Cubo Carne, el palillo y los otros condimentos', 'Agregar el agua y esperar que hierva', 'Añadir el olluco picado y cocinar a fuego medio por 15 minutos', 'Una vez cocidos, agregar el queso cortado en cubos', 'Finalmente, incorporar 1/4 taza deIDEAL® CREMOSITAy mezclar bien', 'Servir acompañado de 1/2 taza de arroz y 1/2 huevo sancochado']


In [ ]:
receta, ingredientes, instrucciones = f_traducir_texto(receta, ingredientes, instrucciones)
print("Receta QUY:", receta)
print("Ingredientes QUY:", ingredientes)
print("Instrucciones QUY:", instrucciones)

Receta QUY: ['Ajiaco de Olluco nisqa']
Ingredientes QUY: ['1 Puka Ch’uñu', '', 'tawa kuchuman kuchusqa', '1 Cuchara Ajo ñut’usqa', '1 Cuchara Palillo', '', 'polvo nisqawan ruwasqa', '1 Unidad de MAGGI® Aycha Kubo', '1 Cuchara Aceite Verdura', '1 vaso yaku', '3 Tazas de Olluco', '', 'kuchusqa', '3 Musuq Quesomanta phatmakuna', '', 'kuchusqa', '1 Lata IDEAL® CREMOSITA nisqamanta', '2 uhu', '1 Pinza de Pimienta', '1 Pincha de Comino', '2 Vaso yanusqa arroz']
Instrucciones QUY: ['Huch’uy mankapi aceiteta ruphaykachispa ch’uñuta, ajota ima q’illuyachina.', 'Chaymantaqa MAGGI Cubo Aychata, kiru hampita, huk condimentokunatapas yapaykusun.', 'Chay yakuta yapaykuspa timpunanta suyana', 'Chaymanqa olluco phatmasqata churaspa chawpi ninapi 15 minutu yanuna.', 'Huk kuti yanusqaña, chaymanqa queso kuchusqa cubos nisqaman.', 'Tukuyninpiqa, 1/4 taza IDEAL® CREMOSITA nisqawan yapaykusun, hinaspa allinta chaqruna.', '1/2 taza arrozwan, 1/2 timpusqa uhuwan kuska haywarina']


## Conversión a Recetas en Quechua

Cargamos los archivos al entorno local de forma manual en la carpeta /content/

In [ ]:
!rm -rf recetas/

Creamos una carpeta en la que se van a almacenar las recetas en quechua

In [ ]:
%mkdir recetas_quy

Estandarizamos los nombres para evitar conflictos con UTF-8

In [ ]:
def rename_files(recipe_filepaths):
    for filepath in recipe_filepaths:
        # Extract the directory and file name
        directory, filename = os.path.split(filepath)

        # Define the new file name
        receta, ingredientes, instrucciones = f_obtener_texto(filepath)
        new_filename = os.path.join(directory, receta[0]+'.txt')

        # Rename the file
        os.rename(filepath, new_filename)

rename_files('recetas_procesadas/')

## Funciones de utilidades para gestionar los archivos .txt
- write_quy_file: Crear archivo quy (quechua) para las recetas
- retrieve_txt_files: Leer la lista de recetas en español

In [158]:
def write_quy_file(receta, ingredientes, instrucciones, output_file):
    with open(output_file, 'w') as file:
        file.write("receta\n" + receta[0] + "\n\n")
        file.write("ingredientes\n")
        for ingrediente in ingredientes:
            file.write(ingrediente + "\n")
        file.write("\ninstrucciones\n")
        for instruccion in instrucciones:
            file.write(instruccion + "\n")

def retrieve_txt_files(folder_path):
    txt_files = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                txt_files.append(os.path.join(root, file))

    return txt_files

Obtenemos la sección de folderes

In [138]:
folder_path = 'recetas_procesadas'
recipe_filepaths = retrieve_txt_files(folder_path)
recipe_filepaths.sort()

In [145]:
start = 69
end = 137
section_recipes = recipe_filepaths[start:end]

Testing write_quy

In [ ]:
write_quy_file(receta, ingredientes, instrucciones, 'recetas_quy/'+receta[0]+'.txt')

## Creación de las recetas en quechua
Creamos los archivos

In [151]:

def create_quy_folder(recipe_filepaths, esp_filepath, quy_filepath):
  for recipe_path in recipe_filepaths:
    receta, ingredientes, instrucciones = f_obtener_texto(recipe_path)
    receta, ingredientes, instrucciones = f_traducir_texto(receta, ingredientes, instrucciones)
    recipe_name = recipe_path.lstrip(esp_filepath)
    quy_recipe_filepath = quy_filepath + recipe_name
    write_quy_file(receta, ingredientes, instrucciones, quy_recipe_filepath)
    print(f"Created file {quy_recipe_filepath}")




In [ ]:
!rm -rf recetas_quy
%mkdir recetas_quy
create_quy_folder(section_recipes, 'recetas_procesadas/', 'recetas_quy/')

Created file recetas_quy/Cau_Cau_de_Pollo.txt
Created file recetas_quy/Cañitas_fritas_rellenas_de_crema.txt
Created file recetas_quy/Cebiche_de_Pollo.txt
Created file recetas_quy/Champiñones_al_ajillo.txt
Created file recetas_quy/Chanfainita.txt
Created file recetas_quy/Charquican.txt
